In [31]:
import torch
import numpy as np
import warnings
from torch.utils.data import Dataset


class Turbofandataset(Dataset):
    def __init__(self, mode='train', dataset=None, rul_result=None):
        self.data = np.loadtxt(fname=dataset, dtype=np.float32)
        # delete useless sensors
        self.data = np.delete(self.data, [5, 9, 10, 14, 20, 22, 23], axis=1)   
        # window size
        self.window_size = 30
        self.sample_num = int(self.data[-1][0])
        self.length = []
        self.mode = mode
        self.max_rul = 120

        if self.mode == 'test' and rul_result is not None:
            self.rul_result = np.loadtxt(fname=rul_result, dtype=np.float32)

        self.x = []
        self.mean_and_coef = []
        self.y = []

        if self.mode == 'train':
            for i in range(1, self.sample_num + 1):
                ind = np.where(self.data[:, 0] == i)
                ind = ind[0]
                # single engine data
                data_temp = self.data[ind, :]
                for j in range(len(data_temp) - self.window_size + 1):
                    self.x.append(data_temp[j: j+self.window_size, 2:])
                    rul = len(data_temp) - self.window_size - j
                    if rul > self.max_rul:
                        rul = self.max_rul
                    self.y.append(rul)

        if self.mode == 'test':
            for i in range(1, self.sample_num + 1):
                ind = np.where(self.data[:, 0] == i)[0]
                data_temp = self.data[ind, :]
                
                if len(data_temp) < self.window_size:
                    data = np.zeros((self.window_size, data_temp.shape[1]))
                    for j in range(data.shape[1]):
                        x_old = np.linspace(0, len(data_temp)-1, len(data_temp), dtype=np.float64)
                        params = np.polyfit(x_old, data_temp[:, j].flatten(), deg=1)
                        k = params[0]
                        b = params[1]
                        x_new = np.linspace(0, self.window_size-1, self.window_size, dtype=np.float64)
                        data[:, j] = (x_new * len(data_temp) / self.window_size * k + b)
                    self.x.append(data[-self.window_size:, 2:])
                else:
                    self.x.append(data_temp[-self.window_size:, 2:])
                rul = self.rul_result[i - 1]
                if rul > self.max_rul:
                    rul = self.max_rul
                self.y.append(rul)

        self.x = np.array(self.x)
        self.y = np.array(self.y)/self.max_rul
        for i in range(len(self.x)):
            one_sample = self.x[i]
            self.mean_and_coef.append(self.fea_extract(one_sample))

        mu = np.mean(self.mean_and_coef, axis=0)
        sigma = np.std(self.mean_and_coef, axis=0)
        eps = 1e-10
        self.mean_and_coef = (self.mean_and_coef - mu) / (sigma + eps)
        self.mean_and_coef = np.array(self.mean_and_coef)

    @staticmethod
    def fea_extract(data):
        fea = []
        x = np.array(range(data.shape[0]))
        for i in range(data.shape[1]):
            fea.append(np.mean(data[:, i]))
            fea.append(np.polyfit(x.flatten(), data[:, i].flatten(), deg=1)[0])
        return fea

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        x_tensor = torch.from_numpy(self.x[index]).to(torch.float32)
        y_tensor = torch.Tensor([self.y[index]]).to(torch.float32)
        handcrafted_features = torch.from_numpy(self.mean_and_coef[index]).to(torch.float32)
        return x_tensor, handcrafted_features, y_tensor


Model

In [32]:
import torch
from torch import nn

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.gru = nn.GRU(batch_first=True, input_size=17, hidden_size=50, num_layers=2)
        self.attention = SelfAttentionBlock(input_size=50, num_heads=5)  # Adjust num_heads as needed
        self.linear = nn.Sequential(
            nn.Linear(in_features=1500, out_features=100),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=100, out_features=10),
            nn.ReLU(inplace=True)
        )
        self.handcrafted = nn.Sequential(
            nn.Linear(in_features=34, out_features=10),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2)
        )

        self.output = nn.Sequential(
            nn.Linear(in_features=20, out_features=1)
        )

    def forward(self, inputs, handcrafted_feature):
        y = self.handcrafted(handcrafted_feature)
        x = self.gru(inputs)
        x = self.attention(x)
        # flatten
        x = x.reshape(-1, 1500)
        x = self.linear(x)
        out = torch.concat((x, y), dim=1)
        out = self.output(out)
        return out


class SelfAttentionBlock(nn.Module):
    def __init__(self, input_size, num_heads=1):
        super(SelfAttentionBlock, self).__init__()
        self.self_attention = nn.MultiheadAttention(embed_dim=input_size, num_heads=num_heads)

    def forward(self, inputs):
        # inputs: (seq_len, batch_size, input_size)
        attention_output, _ = self.self_attention(inputs, inputs, inputs)
        output = inputs + attention_output
        return output

Train

In [33]:
import torch
import numpy as np


class Trainer:
    def __init__(self, model, model_optimizer, print_every, epochs=200, device='cpu', prefix='FD001'):
        self.model = model.to(device)
        self.model_optimizer = model_optimizer
        self.print_every = print_every
        self.epochs = epochs
        self.device = device
        self.criterion = torch.nn.MSELoss()
        self.prefix = prefix

    def train_single_epoch(self, dataloader):
        running_loss = 0

        length = len(dataloader)

        for batch_index, data in enumerate(dataloader, 0):
            inputs, handcrafted_feature, labels = data
            inputs, handcrafted_feature, labels = inputs.to(self.device), handcrafted_feature.to(self.device), labels.to(self.device)
            self.model_optimizer.zero_grad()
            predictions = self.model(inputs, handcrafted_feature)
            loss = self.criterion(predictions, labels)
            running_loss += loss.item()
            loss.backward()

            self.model_optimizer.step()

            if (batch_index + 1) % self.print_every == 0:
                print('batch:{}/{}, loss(avg. on {} batches: {}'.format(batch_index + 1, length, 
                                                                        self.print_every, running_loss / self.print_every, ))
                running_loss = 0

    def train(self, train_loader, test_loader, iteration):
        for epoch in range(self.epochs):
            print('Epoch: {}'.format(epoch + 1))
            self.model.train()
            self.train_single_epoch(train_loader)
            current_score, current_RMSE = self.test(test_loader)
            if epoch == 0:
                best_score = current_score
                best_RMSE = current_RMSE
            else:
                if current_score < best_score:
                    best_score = current_score
                    self.save_checkpoints(iteration + 1, epoch + 1, 'best_score')
                if current_RMSE < best_RMSE:
                    best_RMSE = current_RMSE
                    self.save_checkpoints(iteration + 1, epoch + 1, 'best_RMSE')
        return float(best_score), float(best_RMSE)

    def save_checkpoints(self, iteration, epoch, which_type):
        state = {
            'iter': iteration,
            'epoch': epoch,
            'state_dict': self.model.state_dict(),
            'optim_dict': self.model_optimizer.state_dict()
        }
        torch.save(state, './checkpoints/{}_iteration{}_{}.pth.tar'.format(self.prefix, iteration, which_type))
        print('{}_checkpoints saved successfully!'.format(which_type))

    @staticmethod
    def score(y_true, y_pred):
        score = 0
        y_true = y_true.cpu()
        y_pred = y_pred.cpu()
        for i in range(len(y_pred)):
            if y_true[i] <= y_pred[i]:
                score = score + np.exp(-(y_true[i] - y_pred[i]) / 10.0) - 1
            else:
                score = score + np.exp((y_true[i] - y_pred[i]) / 13.0) - 1
        return score

    def test(self, test_loader):
        score = 0
        loss = 0
        self.model.eval()
        criterion = torch.nn.MSELoss()
        for batch_index, data in enumerate(test_loader, 0):
            with torch.no_grad():
                inputs, handcrafted_feature, labels = data
                inputs, handcrafted_feature, labels = inputs.to(self.device), handcrafted_feature.to(self.device), labels.to(self.device)
                predictions = self.model(inputs, handcrafted_feature)

                score += self.score(labels * 120, predictions * 120)
                loss += criterion(labels * 120, predictions * 120) * len(labels)
        loss = (loss / len(test_loader.dataset)) ** 0.5
        print('test result: score: {}, RMSE: {}'.format(score.item(), loss))
        return score.item(), loss

Main

In [34]:
import torch
import numpy as np
from torch import optim as optim
from torch.utils.data import DataLoader
import warnings
from torch.utils.data import Dataset

if __name__ == '__main__':
    trainset = Turbofandataset(mode='train',
                               dataset='./train_FD001_normed.txt')
    train_loader = DataLoader(dataset=trainset, batch_size=32, shuffle=True, num_workers=0)

    testset = Turbofandataset(mode='test',
                              dataset='./test_FD001_normed.txt',
                              rul_result='./RUL_FD001.txt')
    test_loader = DataLoader(dataset=testset, batch_size=32, shuffle=False, num_workers=0)
    print('Dataset load successfully!')

    best_score_list = []
    best_RMSE_list = []
    for iteration in range(10):
        print('---Iteration: {}---'.format(iteration + 1))
        model = Model()
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        epochs = 20
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        trainer = Trainer(model=model,
                          model_optimizer=optimizer,
                          print_every=50,
                          epochs=epochs,
                          device=device,
                          prefix='FD001')
        best_score, best_RMSE = trainer.train(train_loader, test_loader, iteration)
        best_score_list.append(best_score)
        best_RMSE_list.append(best_RMSE)

    best_score_list = np.array(best_score_list)
    best_RMSE_list = np.array(best_RMSE_list)
    result = np.concatenate((best_score_list, best_RMSE_list)).reshape(2, 10)
    print(result)
    #np.savetxt('./{}_result.txt'.format(trainer.prefix), result, fmt='%.2f')

Dataset load successfully!
---Iteration: 1---
Epoch: 1
batch:50/555, loss(avg. on 50 batches: 0.07601317193359136
batch:100/555, loss(avg. on 50 batches: 0.03384604897350073
batch:150/555, loss(avg. on 50 batches: 0.025956653635948897
batch:200/555, loss(avg. on 50 batches: 0.022361384183168413
batch:250/555, loss(avg. on 50 batches: 0.021376243010163307
batch:300/555, loss(avg. on 50 batches: 0.020391524881124497
batch:350/555, loss(avg. on 50 batches: 0.020019630920141935
batch:400/555, loss(avg. on 50 batches: 0.01870237311348319
batch:450/555, loss(avg. on 50 batches: 0.01749726040288806
batch:500/555, loss(avg. on 50 batches: 0.016404812783002855
batch:550/555, loss(avg. on 50 batches: 0.017127971276640893
test result: score: 259.5659484863281, RMSE: 13.586708068847656
Epoch: 2
batch:50/555, loss(avg. on 50 batches: 0.01696558628231287
batch:100/555, loss(avg. on 50 batches: 0.016480503417551518
batch:150/555, loss(avg. on 50 batches: 0.015615449510514736
batch:200/555, loss(avg. 

batch:450/555, loss(avg. on 50 batches: 0.010444932784885168
batch:500/555, loss(avg. on 50 batches: 0.010429959036409855
batch:550/555, loss(avg. on 50 batches: 0.009684458542615175
test result: score: 265.1774597167969, RMSE: 12.381251335144043
Epoch: 12
batch:50/555, loss(avg. on 50 batches: 0.009406074685975908
batch:100/555, loss(avg. on 50 batches: 0.009642306584864854
batch:150/555, loss(avg. on 50 batches: 0.00938623394817114
batch:200/555, loss(avg. on 50 batches: 0.011284051528200507
batch:250/555, loss(avg. on 50 batches: 0.010402085110545158
batch:300/555, loss(avg. on 50 batches: 0.009614520268514752
batch:350/555, loss(avg. on 50 batches: 0.009021720327436923
batch:400/555, loss(avg. on 50 batches: 0.009488300876691938
batch:450/555, loss(avg. on 50 batches: 0.009514258690178395
batch:500/555, loss(avg. on 50 batches: 0.008898984761908651
batch:550/555, loss(avg. on 50 batches: 0.009189201425760985
test result: score: 195.6531219482422, RMSE: 12.300045013427734
best_score

batch:450/555, loss(avg. on 50 batches: 0.01451918162405491
batch:500/555, loss(avg. on 50 batches: 0.014344906536862254
batch:550/555, loss(avg. on 50 batches: 0.01360931313596666
test result: score: 345.1549377441406, RMSE: 13.506914138793945
best_score_checkpoints saved successfully!
best_RMSE_checkpoints saved successfully!
Epoch: 3
batch:50/555, loss(avg. on 50 batches: 0.015030128005892039
batch:100/555, loss(avg. on 50 batches: 0.015184569954872131
batch:150/555, loss(avg. on 50 batches: 0.013832519883289934
batch:200/555, loss(avg. on 50 batches: 0.01371345586143434
batch:250/555, loss(avg. on 50 batches: 0.013124254764989018
batch:300/555, loss(avg. on 50 batches: 0.013923126244917513
batch:350/555, loss(avg. on 50 batches: 0.013616759264841675
batch:400/555, loss(avg. on 50 batches: 0.013728516632691026
batch:450/555, loss(avg. on 50 batches: 0.01392249584197998
batch:500/555, loss(avg. on 50 batches: 0.012588686430826784
batch:550/555, loss(avg. on 50 batches: 0.012722253669

batch:100/555, loss(avg. on 50 batches: 0.009419370936229824
batch:150/555, loss(avg. on 50 batches: 0.008907207632437348
batch:200/555, loss(avg. on 50 batches: 0.008501422675326466
batch:250/555, loss(avg. on 50 batches: 0.009844438517466187
batch:300/555, loss(avg. on 50 batches: 0.00981379627250135
batch:350/555, loss(avg. on 50 batches: 0.008780291909351945
batch:400/555, loss(avg. on 50 batches: 0.009350609867833554
batch:450/555, loss(avg. on 50 batches: 0.008734412044286727
batch:500/555, loss(avg. on 50 batches: 0.010059953248128296
batch:550/555, loss(avg. on 50 batches: 0.00911475395783782
test result: score: 236.57444763183594, RMSE: 12.41345500946045
Epoch: 14
batch:50/555, loss(avg. on 50 batches: 0.009698096830397844
batch:100/555, loss(avg. on 50 batches: 0.007796493633650243
batch:150/555, loss(avg. on 50 batches: 0.009152632104232908
batch:200/555, loss(avg. on 50 batches: 0.008439235999248921
batch:250/555, loss(avg. on 50 batches: 0.009179415507242083
batch:300/555,

batch:50/555, loss(avg. on 50 batches: 0.012299183160066604
batch:100/555, loss(avg. on 50 batches: 0.01284768046811223
batch:150/555, loss(avg. on 50 batches: 0.013142130738124251
batch:200/555, loss(avg. on 50 batches: 0.0113382366951555
batch:250/555, loss(avg. on 50 batches: 0.012480414416640996
batch:300/555, loss(avg. on 50 batches: 0.012912103831768036
batch:350/555, loss(avg. on 50 batches: 0.01307509385049343
batch:400/555, loss(avg. on 50 batches: 0.012125019067898393
batch:450/555, loss(avg. on 50 batches: 0.012549283634871244
batch:500/555, loss(avg. on 50 batches: 0.011801557932049036
batch:550/555, loss(avg. on 50 batches: 0.01292679090052843
test result: score: 248.53424072265625, RMSE: 12.647632598876953
best_RMSE_checkpoints saved successfully!
Epoch: 5
batch:50/555, loss(avg. on 50 batches: 0.011853691134601831
batch:100/555, loss(avg. on 50 batches: 0.011656479444354773
batch:150/555, loss(avg. on 50 batches: 0.012307369969785214
batch:200/555, loss(avg. on 50 batche

...

batch:100/555, loss(avg. on 50 batches: 0.009535983437672257
batch:150/555, loss(avg. on 50 batches: 0.009270337671041488
batch:200/555, loss(avg. on 50 batches: 0.009716735258698463
batch:250/555, loss(avg. on 50 batches: 0.01008066102862358
batch:300/555, loss(avg. on 50 batches: 0.009623860763385892
batch:350/555, loss(avg. on 50 batches: 0.009485726822167634
batch:400/555, loss(avg. on 50 batches: 0.009003308927640319
batch:450/555, loss(avg. on 50 batches: 0.00829434079118073
batch:500/555, loss(avg. on 50 batches: 0.009575050491839647
batch:550/555, loss(avg. on 50 batches: 0.010310571901500224
test result: score: 210.78916931152344, RMSE: 12.43464183807373
best_score_checkpoints saved successfully!
Epoch: 12
batch:50/555, loss(avg. on 50 batches: 0.010504256142303348
batch:100/555, loss(avg. on 50 batches: 0.009470103187486529
batch:150/555, loss(avg. on 50 batches: 0.00870318171568215
batch:200/555, loss(avg. on 50 batches: 0.009244932867586613
batch:250/555, loss(avg. on 50 ba

batch:550/555, loss(avg. on 50 batches: 0.01869950653985143
test result: score: 271.72174072265625, RMSE: 14.921148300170898
Epoch: 2
batch:50/555, loss(avg. on 50 batches: 0.01590434062294662
batch:100/555, loss(avg. on 50 batches: 0.01553546329960227
batch:150/555, loss(avg. on 50 batches: 0.014921146742999554
batch:200/555, loss(avg. on 50 batches: 0.014386917175725102
batch:250/555, loss(avg. on 50 batches: 0.014663646146655083
batch:300/555, loss(avg. on 50 batches: 0.01416696572676301
batch:350/555, loss(avg. on 50 batches: 0.01498180853202939
batch:400/555, loss(avg. on 50 batches: 0.013862011628225446
batch:450/555, loss(avg. on 50 batches: 0.014530675541609526
batch:500/555, loss(avg. on 50 batches: 0.013228037655353546
batch:550/555, loss(avg. on 50 batches: 0.013791037080809474
test result: score: 295.3391418457031, RMSE: 13.56969165802002
best_RMSE_checkpoints saved successfully!
Epoch: 3
batch:50/555, loss(avg. on 50 batches: 0.012721947617828846
batch:100/555, loss(avg. o

batch:250/555, loss(avg. on 50 batches: 0.008872403996065259
batch:300/555, loss(avg. on 50 batches: 0.008845731681212783
batch:350/555, loss(avg. on 50 batches: 0.009146059923805296
batch:400/555, loss(avg. on 50 batches: 0.009411843842826784
batch:450/555, loss(avg. on 50 batches: 0.008520026942715048
batch:500/555, loss(avg. on 50 batches: 0.009713334571570158
batch:550/555, loss(avg. on 50 batches: 0.0096271045319736
test result: score: 224.94110107421875, RMSE: 12.351386070251465
Epoch: 13
batch:50/555, loss(avg. on 50 batches: 0.009258796228095889
batch:100/555, loss(avg. on 50 batches: 0.009317482821643353
batch:150/555, loss(avg. on 50 batches: 0.008536097519099712
batch:200/555, loss(avg. on 50 batches: 0.008714481219649316
batch:250/555, loss(avg. on 50 batches: 0.009122130814939738
batch:300/555, loss(avg. on 50 batches: 0.009530552001670002
batch:350/555, loss(avg. on 50 batches: 0.009099721275269985
batch:400/555, loss(avg. on 50 batches: 0.009292437639087438
batch:450/555

batch:200/555, loss(avg. on 50 batches: 0.014116693083196878
batch:250/555, loss(avg. on 50 batches: 0.013055361453443765
batch:300/555, loss(avg. on 50 batches: 0.012883618017658591
batch:350/555, loss(avg. on 50 batches: 0.012988544423133136
batch:400/555, loss(avg. on 50 batches: 0.012873758049681782
batch:450/555, loss(avg. on 50 batches: 0.013784344671294094
batch:500/555, loss(avg. on 50 batches: 0.012564469492062927
batch:550/555, loss(avg. on 50 batches: 0.012685222495347261
test result: score: 240.05502319335938, RMSE: 14.0746431350708
best_score_checkpoints saved successfully!
Epoch: 4
batch:50/555, loss(avg. on 50 batches: 0.013018746050074697
batch:100/555, loss(avg. on 50 batches: 0.01213103149086237
batch:150/555, loss(avg. on 50 batches: 0.012600466459989548
batch:200/555, loss(avg. on 50 batches: 0.01255105678923428
batch:250/555, loss(avg. on 50 batches: 0.011432876512408256
batch:300/555, loss(avg. on 50 batches: 0.011941507272422314
batch:350/555, loss(avg. on 50 bat

batch:100/555, loss(avg. on 50 batches: 0.008175665526650847
batch:150/555, loss(avg. on 50 batches: 0.009026911412365734
batch:200/555, loss(avg. on 50 batches: 0.008919702982529998
batch:250/555, loss(avg. on 50 batches: 0.009221778875216842
batch:300/555, loss(avg. on 50 batches: 0.009578384440392256
batch:350/555, loss(avg. on 50 batches: 0.009294414259493351
batch:400/555, loss(avg. on 50 batches: 0.009894589916802943
batch:450/555, loss(avg. on 50 batches: 0.009215198773890734
batch:500/555, loss(avg. on 50 batches: 0.008654409125447272
batch:550/555, loss(avg. on 50 batches: 0.00877032121643424
test result: score: 208.10279846191406, RMSE: 13.440435409545898
best_score_checkpoints saved successfully!
Epoch: 15
batch:50/555, loss(avg. on 50 batches: 0.009448749623261391
batch:100/555, loss(avg. on 50 batches: 0.008457124754786492
batch:150/555, loss(avg. on 50 batches: 0.008733368031680584
batch:200/555, loss(avg. on 50 batches: 0.008009918057359755
batch:250/555, loss(avg. on 50